<a href="https://colab.research.google.com/github/saurabhkr36/Network_Congestion_in_telecom_industry/blob/master/Network_congestion_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import matthews_corrcoef

In [2]:
trainset=pd.read_csv('train_upd.csv')
testset=pd.read_csv('test_upd.csv')

In [3]:
trainset.describe()

,cell_name,4G_rat,par_year,par_month,par_day,par_hour,par_min,subscriber_count,web_browsing_total_bytes,video_total_bytes,social_ntwrking_bytes,cloud_computing_total_bytes,web_security_total_bytes,gaming_total_bytes,health_total_bytes,communication_total_bytes,file_sharing_total_bytes,remote_access_total_bytes,photo_sharing_total_bytes,software_dwnld_total_bytes,marketplace_total_bytes,storage_services_total_bytes,audio_total_bytes,location_services_total_bytes,presence_total_bytes,advertisement_total_bytes,system_total_bytes,voip_total_bytes,speedtest_total_bytes,email_total_bytes,weather_total_bytes,media_total_bytes,mms_total_bytes,others_total_bytes,beam_direction,cell_range,tilt
count,7.856000e+04,78560.000000,78560.0,78560.0,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000,78560.000000
mean,2.785900e+09,0.499173,2018.0,12.0,15.503831,11.514588,33.061736,571.187131,8091.673002,8293.335871,8358.866204,628.757536,72.102864,71.150433,72.123994,4287.038607,18.478933,18.104697,39.435120,38.865937,32.184891,70.354595,32.444959,70.603513,4224.867541,70.688480,70.308007,70.619539,70.689779,70.987156,70.945481,70.594857,70.719297,69.728182,89.991701,4.492146,3.503055
std,1.604667e+09,0.500002,0.0,0.0,8.634375,6.934351,16.535863,1080.263507,19145.523831,19462.783122,19498.255773,1255.947419,118.254690,115.541885,117.917609,9836.646337,25.742650,25.286846,60.087207,60.325677,48.480249,115.258152,49.054204,117.304376,9610.445883,116.585996,116.221184,117.255264,118.102336,117.821916,117.134609,117.647164,118.032907,114.619303,17.638370,1.709296,1.115255
min,8.737700e+04,0.000000,2018.0,12.0,1.000000,0.000000,5.000000,1.000000,3.000000,5.000000,7.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60.000000,2.000000,2.000000
25%,1.395260e+09,0.000000,2018.0,12.0,8.000000,5.000000,20.000000,71.000000,431.000000,471.000000,469.000000,69.000000,14.000000,14.000000,13.000000,275.000000,5.000000,4.000000,9.000000,8.000000,7.000000,13.000000,7.000000,13.000000,270.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,75.000000,3.000000,3.000000
50%,2.786354e+09,0.000000,2018.0,12.0,16.000000,12.000000,35.000000,187.000000,1545.500000,1672.500000,1682.000000,189.000000,31.000000,31.000000,31.000000,914.000000,10.000000,10.000000,19.000000,18.000000,15.000000,30.000000,16.000000,30.000000,903.500000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,90.000000,4.000000,4.000000
75%,4.172220e+09,1.000000,2018.0,12.0,23.000000,18.000000,45.000000,568.000000,6565.000000,6884.250000,7008.500000,593.000000,77.000000,76.000000,77.000000,3615.000000,21.000000,21.000000,43.000000,42.000000,36.000000,75.000000,36.000000,75.000000,3554.250000,75.000000,75.000000,75.000000,75.000000,75.000000,76.000000,74.000000,75.000000,75.000000,105.000000,6.000000,4.000000
max,5.555540e+09,1.000000,2018.0,12.0,30.000000,23.000000,60.000000,16570.000000,311008.000000,331271.000000,303204.000000,18901.000000,1666.000000,1642.000000,1575.000000,160324.000000,329.000000,327.000000,754.000000,841.000000,588.000000,1568.000000,654.000000,1657.000000,155129.000000,1661.000000,1553.000000,1560.000000,1612.000000,1563.000000,1680.000000,1661.000000,1611.000000,1600.000000,120.000000,7.000000,5.000000


In [4]:
testset.describe()

,cell_name,4G_rat,par_year,par_month,par_day,par_hour,par_min,subscriber_count,web_browsing_total_bytes,video_total_bytes,social_ntwrking_bytes,cloud_computing_total_bytes,web_security_total_bytes,gaming_total_bytes,health_total_bytes,communication_total_bytes,file_sharing_total_bytes,remote_access_total_bytes,photo_sharing_total_bytes,software_dwnld_total_bytes,marketplace_total_bytes,storage_services_total_bytes,audio_total_bytes,location_services_total_bytes,presence_total_bytes,advertisement_total_bytes,system_total_bytes,voip_total_bytes,speedtest_total_bytes,email_total_bytes,weather_total_bytes,media_total_bytes,mms_total_bytes,others_total_bytes,beam_direction,cell_range,tilt
count,2.630500e+04,26305.000000,26305.0,26305.0,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.000000,26305.00000
mean,2.790329e+09,0.498232,2018.0,12.0,15.505797,11.452195,32.904391,589.943813,8148.133093,8442.448926,8608.163961,637.613496,73.630146,71.817829,72.841893,4225.167649,18.747538,18.316860,39.949667,39.977685,32.904239,70.839384,32.096788,71.951036,4342.577191,72.141190,71.711500,70.524387,70.617297,72.146436,70.471355,71.170348,72.029348,70.181905,89.853830,4.491846,3.51112
std,1.595260e+09,0.500006,0.0,0.0,8.651438,6.893525,16.565754,1136.003055,19521.376974,19526.321292,19907.852853,1280.492464,122.299048,117.808955,119.925433,9663.335207,26.477269,25.668914,60.919895,62.565050,50.557818,116.940056,47.544632,117.920428,9914.512447,119.982656,120.697444,115.293322,116.672562,118.543565,114.689548,117.568749,118.309746,116.241109,17.570263,1.702993,1.11164
min,4.476350e+05,0.000000,2018.0,12.0,1.000000,0.000000,5.000000,1.000000,6.000000,5.000000,6.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60.000000,2.000000,2.00000
25%,1.416926e+09,0.000000,2018.0,12.0,8.000000,6.000000,20.000000,72.000000,441.000000,473.000000,472.000000,70.000000,14.000000,14.000000,13.000000,277.000000,5.000000,4.000000,8.000000,8.000000,7.000000,13.000000,7.000000,13.000000,278.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,75.000000,3.000000,3.00000
50%,2.776931e+09,0.000000,2018.0,12.0,16.000000,11.000000,35.000000,189.000000,1561.000000,1687.000000,1682.000000,190.000000,31.000000,31.000000,31.000000,923.000000,10.000000,10.000000,19.000000,19.000000,16.000000,30.000000,16.000000,30.000000,929.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,31.000000,30.000000,90.000000,4.000000,4.00000
75%,4.181104e+09,1.000000,2018.0,12.0,23.000000,17.000000,45.000000,579.000000,6558.000000,6981.000000,7184.000000,604.000000,78.000000,77.000000,78.000000,3547.000000,21.000000,21.000000,44.000000,44.000000,36.000000,75.000000,36.000000,76.000000,3605.000000,76.000000,75.000000,75.000000,75.000000,78.000000,75.000000,76.000000,77.000000,75.000000,105.000000,6.000000,4.00000
max,5.554979e+09,1.000000,2018.0,12.0,30.000000,23.000000,60.000000,14572.000000,305413.000000,294798.000000,307922.000000,18093.000000,1501.000000,1590.000000,1622.000000,153605.000000,324.000000,305.000000,747.000000,709.000000,629.000000,1553.000000,586.000000,1429.000000,151948.000000,1586.000000,1646.000000,1519.000000,1469.000000,1481.000000,1535.000000,1414.000000,1453.000000,1602.000000,120.000000,7.000000,5.00000


In [5]:
def enc(k):

   if k=='4G_RAN_CONGESTION':
      return 1

   elif k=='4G_BACKHAUL_CONGESTION':
      return 2

   elif k=='3G_BACKHAUL_CONGESTION':
      return 3

   else :
      return 0

In [6]:
trainset['Congestion_Type'] = trainset['Congestion_Type'].apply(enc)

y=trainset['Congestion_Type']

In [7]:
trainset.drop(['cell_name','Congestion_Type','par_year','par_month'],axis=1,inplace=True)

In [8]:
testset.drop(['cell_name','par_year','par_month'],axis=1,inplace=True)

In [9]:
dummies=pd.get_dummies(trainset['ran_vendor'],prefix='ran_vendor')
trainset=pd.concat([trainset,dummies],axis=1)
trainset.drop(['ran_vendor'],axis=1,inplace=True)

In [10]:
dummies=pd.get_dummies(testset['ran_vendor'],prefix='ran_vendor')
testset=pd.concat([testset,dummies],axis=1)
testset.drop(['ran_vendor'],axis=1,inplace=True)

In [11]:
trainset.head()

,4G_rat,par_day,par_hour,par_min,subscriber_count,web_browsing_total_bytes,video_total_bytes,social_ntwrking_bytes,cloud_computing_total_bytes,web_security_total_bytes,gaming_total_bytes,health_total_bytes,communication_total_bytes,file_sharing_total_bytes,remote_access_total_bytes,photo_sharing_total_bytes,software_dwnld_total_bytes,marketplace_total_bytes,storage_services_total_bytes,audio_total_bytes,location_services_total_bytes,presence_total_bytes,advertisement_total_bytes,system_total_bytes,voip_total_bytes,speedtest_total_bytes,email_total_bytes,weather_total_bytes,media_total_bytes,mms_total_bytes,others_total_bytes,beam_direction,cell_range,tilt,ran_vendor_ERICSSON,ran_vendor_HUAWEI,ran_vendor_NOKIA
0,1,15,0,50,152,315,59012,2266,1034,113,395,68,10614,2,16,9,39,2,9,34,55,11888,88,15,15,10,10,33,46,39,103,69,6,2,1,0,0
1,1,7,20,5,54,1871,334,4530,63,29,14,11,135,2,5,14,14,6,17,24,2,239,46,6,18,8,22,20,7,10,21,93,7,5,0,0,1
2,1,5,16,35,277,887,945,23753,171,18,6,52,3818,4,8,36,33,13,45,10,3,15763,8,40,145,23,14,91,49,147,159,77,6,3,1,0,0
3,0,16,21,60,51,8577,494,7956,66,60,19,14,2114,18,10,98,8,2,14,80,40,153,12,7,34,3,10,63,42,75,35,95,7,2,0,1,0
4,1,4,6,60,117,2084,1056,7244,232,59,90,19,605,16,89,14,36,70,22,41,839,962,115,42,272,4,67,59,91,197,316,112,3,5,0,1,0


In [12]:
testset.head()

,4G_rat,par_day,par_hour,par_min,subscriber_count,web_browsing_total_bytes,video_total_bytes,social_ntwrking_bytes,cloud_computing_total_bytes,web_security_total_bytes,gaming_total_bytes,health_total_bytes,communication_total_bytes,file_sharing_total_bytes,remote_access_total_bytes,photo_sharing_total_bytes,software_dwnld_total_bytes,marketplace_total_bytes,storage_services_total_bytes,audio_total_bytes,location_services_total_bytes,presence_total_bytes,advertisement_total_bytes,system_total_bytes,voip_total_bytes,speedtest_total_bytes,email_total_bytes,weather_total_bytes,media_total_bytes,mms_total_bytes,others_total_bytes,beam_direction,cell_range,tilt,ran_vendor_ERICSSON,ran_vendor_HUAWEI,ran_vendor_NOKIA
0,1,8,23,30,547,209,25799,16664,1395,165,108,123,419,22,11,27,8,36,102,4,61,460,24,50,7,5,8,42,12,15,23,120,3,3,1,0,0
1,1,20,5,20,12,822,3865,66,253,80,18,22,247,2,12,23,8,6,95,1,21,428,20,29,18,3,341,77,5,21,155,91,6,3,0,1,0
2,1,15,13,40,1747,10755,1711,18555,187,17,137,21,15,152,54,54,8,55,41,99,117,915,13,80,17,40,707,204,45,280,131,63,4,5,0,1,0
3,1,7,13,50,5,7068,190,21105,166,12,6,8,369,4,34,3,3,10,10,34,27,129,19,2,15,7,1,16,4,25,3,61,4,5,0,1,0
4,1,11,7,50,3298,1052,10151,32412,47,184,125,171,1273,1,2,34,7,14,71,10,198,343,90,15,316,380,7,18,15,45,8,61,7,5,1,0,0


In [13]:
scaler = StandardScaler()
scaler.fit(trainset)
trainset=scaler.transform(trainset)

In [14]:
scaler2 = StandardScaler()
scaler2.fit(testset)
testset=scaler2.transform(testset)

In [15]:
len(trainset)

78560

In [16]:
X_train,X_dev,y_train,y_dev=train_test_split(trainset,y,test_size=0.25,random_state=0)

In [17]:
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
#classifier.fit(trainset, y)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [18]:
y_pred_dev = classifier.predict(X_dev)

In [19]:
#y_pred_test=classifier.predict(testset)

In [20]:
from sklearn.metrics import accuracy_score
acc_dev=accuracy_score(y_dev,y_pred_dev)


In [21]:
def mat(model):
   y_pred = model.predict(X_dev)
   dev=pd.Series.tolist(y_dev)
   pred=np.ndarray.tolist(y_pred)
   #print(test)
   p=matthews_corrcoef(dev,pred)
   return p

In [22]:
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, y_train)
scores = cross_val_score(model, trainset, y, cv=10)
print("Cross-Validation Score of model in training set: ",scores)
y_pred_dev=model.predict(X_dev)
y_pred_train=model.predict(X_train)
#print(np.ndarray.tolist(y_pred))
mcc=mat(model)
print("matthews correlation coefficient = ",mcc)
dev_set_acc=accuracy_score(y_dev, y_pred_dev, normalize=True, sample_weight=None)
train_set_acc=accuracy_score(y_train, y_pred_train, normalize=True, sample_weight=None)
print("Development set Accuracy = "+str(dev_set_acc*100)+"%")
print("training set Accuracy = "+str(train_set_acc*100)+"%")

Cross-Validation Score of model in training set:  [0.57815682 0.5785387  0.58910387 0.5871945  0.59063136 0.58681263
 0.58897658 0.58897658 0.5857943  0.58668534]
matthews correlation coefficient =  0.4666895536520587
Development set Accuracy = 58.28920570264766%
training set Accuracy = 79.62491513917176%


In [23]:
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(trainset, y)
y_pred_test=classifier.predict(testset)

In [24]:
y_pred_test=pd.DataFrame(y_pred_test)
y_pred_test.to_excel('Output.xlsx')